# Import modules
CDC data set description link [here](https://wwwn.cdc.gov/nchs/nhanes/search/datapage.aspx?Component=Questionnaire&CycleBeginYear=2015)

In [1]:
import pdb
import glob
import pickle

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier, AdaBoostClassifier, AdaBoostRegressor
from sklearn.svm import SVC
from sklearn import svm
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.feature_selection import mutual_info_classif, mutual_info_regression
from sklearn import metrics
from sklearn.metrics import classification_report
from sklearn.decomposition import PCA
from sklearn.model_selection import StratifiedKFold, GridSearchCV, train_test_split
from sklearn.pipeline import Pipeline

import nhanes as nhanes

%matplotlib notebook

## Settings

In [2]:
DATA_PATH = 'C:/Users/allen/Documents/Git-Repos/Opportunistic/CDC/NHANES/'
DATASET = 'cancer'

### Note: 
The code below loads each dataset: dataset_features, dataset_targets

Here, all datasets are defined explicitly (see nhanes.py).
*Caution* Very long computation, proceed with care

In [3]:
ds = nhanes.Dataset(DATA_PATH)
ds.load_cancer()
n_fe = ds.features.shape[1]
n_classes = 2

Processing: Questionnaire\RHQ_D.XPT                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     

Processing: Examination\AUXTYM_G.XPT                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    

Processing: Laboratory\L39EPP_C.XPT                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     

Processing: Examination\CVX_C.XPT                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       

Processing: Examination\BMX_C.XPT

...

Processing: Questionnaire\PFQ_I.XPT

Processing: Questionnaire\IMQ_E.XPT                                                                                

Processing: Questionnaire\ACQ.XPT

Processing: Questionnaire\SMQ_H.XPT

Processing: Questionnaire\WHQ_H.XPT

Extract the features and targets.

In [79]:
features = ds.features.values
targets = ds.targets.values

indx = np.argwhere(targets != 3)
dataset_features = features[indx.flatten()]
dataset_targets = targets[indx.flatten()]

dataset_features = pd.DataFrame(dataset_features, columns=ds.features.columns)
dataset_targets = pd.DataFrame(dataset_targets, columns=ds.targets.columns, dtype='float64')
print(dataset_features.shape)
print(dataset_targets.shape)
# Pre-fix
# dataset_features = ds.features
# dataset_targets = ds.targets

(98908, 168)
(98908, 1)


## Preprocessing of Data

### Calculate feature importance

In [ ]:
dataset_features_sel = dataset_features.copy()
importances = []
num_trees = 5

for i in range(num_trees):
    clf = ExtraTreesClassifier(n_estimators=50)
    clf = clf.fit(dataset_features_sel.values, dataset_targets.values.ravel())
    importances.append(clf.feature_importances_)
    print('0% [',(i+1)*'.',(num_trees-i-1)*'-','] 100%', end='\r')

print('\nDone')

#### Drop Threshhold helper function

In [ ]:
def drop_threshold(threshold, df_data, df_threshold):
    '''
    Sort a 1xn pandas dataframe df_data, return only 
    up to sum of threshold value indicies in df_threshhold
    if threshold is <= 1. If threshold >= 1, return the top 
    threshold number of columns.
    '''
    mi = df_threshold # mutual information dataframe
    df = df_data # data containing dataframe
    to_keep = []
    mi = mi.sort_values(by=0, axis=1, ascending=False) #sorts the values descending
    # print(mi)
    var_counter = 0
    col_counter = 0
    if(threshold <=1):
        for col in mi:
            var_counter += mi[col].iloc[0]
            col_counter += 1
            to_keep.append(col)
            if(var_counter >= threshold and threshold <= 1):
                break
            elif(col_counter >= threshold and threshold > 1):
                break
        if(len(to_keep) > 0):
            return pd.DataFrame(df[to_keep].copy(), dtype='float64')
        else:
            raise NotImplementedError

#### Drop low importance features

In [ ]:
importances_mean = np.array(importances)
importances_mean = np.mean(importances_mean, axis=0)
importances_mean = importances_mean.reshape((1, len(importances_mean)))
importances_mean = pd.DataFrame(importances_mean, columns=ds.features.columns, dtype='float64')
importances_mean = importances_mean.sort_values(by=0, axis=1, ascending=False)

gfs = 29 # number features to show 
df_importances = drop_threshold(0.60, dataset_features_sel, importances_mean)
print(df_importances.shape)

# Visualize Importance:
idx = [i for i in range(len(importances_mean.columns))]
cols = [i for i in importances_mean.columns]
# print(importances_mean.values[0][0:gfs])
plt.bar(idx[0:gfs], importances_mean.values[0][0:gfs])
plt.xticks(idx[0:gfs], importances_mean.columns[0:gfs], rotation='vertical')
plt.xlabel('Features')
plt.ylabel('Importance')
plt.show()


#### Seperate one-hot encoded, discrete, and continuous features

In [ ]:
# onehot encoded
dataset_features_onehot = df_importances.filter(regex=(".*#.*"))
# Continuous 
dataset_features_cont = df_importances.drop(columns=df_importances.filter(regex=(".*#.*")))


print("Original shape: %s" %(df_importances.shape,))
print("Continuous features shape: %s" %(dataset_features_cont.shape,))
print("Categorical features shape: %s "%(dataset_features_onehot.shape,))

#### Calculate PCA to drop features

In [ ]:
# dfs = [dataset_features_cont.copy(), dataset_features_disc.copy(), dataset_features_onehot.copy()]
dfs_out = [dataset_features_cont.copy(), dataset_features_onehot.copy()]
pca_dfs = []
gfs = 15 # Number of features to graph
gfs -= 1
i = 0
for df in dfs_out:    
#     print(df.shape)
    pca = PCA() # no args, so keep all columns, n_components=2
    pca.fit(df)
    var_ratio = pca.explained_variance_ratio_
    var_ratio = var_ratio.reshape((1, len(var_ratio)))
    pca_dfs.append(pd.DataFrame(data=var_ratio, columns=df.columns))
    # Visualize PCA:
    idx = [i for i in range(len(df.columns))]
    cols = [i for i in df.columns]
#     plt.subplot(len(dfs_out),1,i+1)
#     plt.subplots_adjust(hspace = 1, top=1, bottom=.3)
    plt.bar(idx[0:gfs], pca.explained_variance_ratio_[0:gfs])
    plt.xticks(idx[0:gfs], df.columns[0:gfs], rotation='vertical')
    plt.xlabel('Features')
    plt.ylabel('Variance')
    i+=1
    plt.show()

# print(pca_dfs.shape)


#### Remove columns that account for too little of the variance

In [ ]:
pca_dfs_out = []

for i in range(len(dfs_out)):
    pca_dfs_out.append(drop_threshold(0.95, dfs_out[i] ,pca_dfs[i]))

for df in pca_dfs_out:
    print(df.shape)

df_final = pd.concat(pca_dfs_out, axis=1)
print(df_final.shape)

print('\nFinal list is %s features' %(df_final.shape[1]))
print(df_final.columns)
# for i in df_final.columns:
#     print(i)

## Train/Test Separation

In [ ]:
perm = np.random.permutation(dataset_targets.shape[0])
try:
    dataset_targets_vals = np.ravel(dataset_targets.values) # unroll the column to the correct vector shape
except:
    print("Ravel already performed on dataset_targets.values")
dataset_features = df_final.values[perm]
dataset_targets = dataset_targets_vals[perm]

print("dataset_features Shape: %s, dataset_targets Shape: %s" % (dataset_features.shape, dataset_targets.shape))

def get_batch(n_size, phase):
    # select indices
    n_samples = dataset_features.shape[0]
    n_classes = int(dataset_targets.max() + 1)
    if phase == 'test':
        inds_sel = np.arange(0, int(n_samples*0.15), 1)
    elif phase == 'validation':
        n_samples = dataset_features.shape[0]
        inds_sel = np.arange(int(n_samples*0.15), int(n_samples*0.30), 1)
    elif phase == 'train':
        n_samples = dataset_features.shape[0]
        inds_sel = np.arange(int(n_samples*0.30), n_samples, 1)
    else:
        raise NotImplementedError
    inds_sel = np.random.permutation(inds_sel)
    batch_inds = []
    for cl in range(n_classes):
        inds_cl = inds_sel[dataset_targets[inds_sel] == cl]
        batch_inds.extend(inds_cl[:n_size//n_classes])
    batch_inds = np.random.permutation(batch_inds)
    
    return dataset_features[batch_inds], dataset_targets[batch_inds]

## Validation
First run for all options, this is a very **long running** calculation

In [ ]:
# Create first pipeline for base without reducing features.
clf_all = None
did_search = True
if not did_search:
    pipe = Pipeline([('classifier' , RandomForestClassifier())])

    # Create param grid.

    param_grid = [
        {'classifier' : [LogisticRegression()],
         'classifier__penalty' : ['l1', 'l2'],
         'classifier__solver' : ['liblinear'],
        'classifier__C' : np.logspace(-4, 4, 20)},
        {'classifier' : [DecisionTreeClassifier()],
        'classifier__criterion' : ['gini', 'entropy']},
        {'classifier' : [SVC()],
         'classifier__kernel':('linear', 'rbf'),
         'classifier__gamma' : ['auto', 'scale'],
         'classifier__C':[0.001, 0.01, 0.1, 1, 10]},
        {'classifier':[AdaBoostClassifier()],
        'classifier__n_estimators' : list(range(30,81,10)),
        'classifier__algorithm' : ['SAMME', 'SAMME.R']},
        {'classifier':[RandomForestClassifier()],
         'classifier__bootstrap': [False, True],
         'classifier__n_estimators': [80,90, 100, 110, 130],
         'classifier__max_features': [0.6, 0.65, 0.7, 0.73, 0.75, 0.78, 0.8],
         'classifier__min_samples_leaf': [10, 12, 14],
         'classifier__min_samples_split': [3, 5, 7]
        },
        
    ]
        
    # Create grid search object

    clf_all = GridSearchCV(pipe, param_grid = param_grid, cv = 5, verbose=True, n_jobs=-1, return_train_score=True)
    # Fit on data
    features_val, targets_val = get_batch(n_size=5000, phase='validation')
    best_clf_all = clf_all.fit(features_val, targets_val)
    print(clf_all.best_params_)

**Output from Grid Search** :: Done 3520 out of 3520 | elapsed: 28.5min finished

{'classifier': RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features=0.78, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=12, min_samples_split=5,
            min_weight_fraction_leaf=0.0, n_estimators=90, n_jobs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False), 'classifier__bootstrap': True, 'classifier__max_features': 0.78, 'classifier__min_samples_leaf': 12, 'classifier__min_samples_split': 5, 'classifier__n_estimators': 90}

In [ ]:
# Resluts:
if (not did_search) and (clf_all is not None):
    with open('gridsearch.pickle', 'wb') as f:
        pickle.dump(clf_all, f)
        print("clf_all saved as gridsearch.pickle.")
elif clf_all is None:
    with open('gridsearch.pickle', 'rb') as f:
        clf_all = pickle.load(f)
        print("clf_all loaded from gridsearch.pickle.")
else:
    print("clf_all is already loaded.")

results = pd.DataFrame(clf_all.cv_results_)
# print(results.head(2))
# print(results.columns)

## Classification

In [ ]:
accumilator = []
reports = []
num_tests = 2
for i in range(num_tests):
    features_trn, targets_trn = get_batch(n_size=5000, phase='train')
    features_tst, targets_tst = get_batch(n_size=1000, phase='test')

    # base = LogisticRegression(C=0.23357214690901212, class_weight=None, dual=False,
    #           fit_intercept=True, intercept_scaling=1, max_iter=100,
    #           multi_class='warn', n_jobs=None, penalty='l1', random_state=None,
    #           solver='liblinear', tol=0.0001, verbose=0, warm_start=False)
    # clf_adb = AdaBoostRegressor(base_estimator=base, n_estimators=60, loss='linear')

    clf_adb = RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                max_depth=None, max_features=0.65, max_leaf_nodes=None,
                min_impurity_decrease=0.0, min_impurity_split=None,
                min_samples_leaf=14, min_samples_split=7,
                min_weight_fraction_leaf=0.0, n_estimators=90, n_jobs=None,
                oob_score=False, random_state=None, verbose=0,
                warm_start=False)

    # clf_adb = clf_all.best_estimator_

    clf_adb.fit(features_trn, targets_trn)
    preds_tst = clf_adb.predict(features_tst)
    accu = np.mean(preds_tst==targets_tst)
    accumilator.append(accu)
    reports.append(pd.DataFrame(classification_report(targets_tst, preds_tst, output_dict=True)))
    print('0% [',(i+1)*'.',(num_tests-i-1)*'-','] 100%', end='\r')

accumilator = np.array(accumilator)
reports = pd.concat(reports).groupby(level=0).mean()
print('accu_tst_ADB_mean', accumilator.mean())
print('accu_tst_ADB', accu)
# print(classification_report(targets_tst, preds_tst))
print("%s, %s, %s, %s" % (
    accumilator.mean(), 
    reports.loc['f1-score', 'weighted avg'],
    reports.loc['precision', 'weighted avg'],
    reports.loc['recall', 'weighted avg']))
fpr, tpr, thresholds = metrics.roc_curve(targets_tst, preds_tst)
print("%s, %s" %(fpr,tpr))
plt.plot(fpr, tpr, label='ROC curve (area = %0.2f)' % metrics.auc(fpr, tpr))
plt.plot([0, 1], [0, 1], color='navy', linestyle='--')
plt.xlabel('FPR')
plt.ylabel('TPR')
plt.legend(loc="lower right")
plt.show()


# clf_adb = clf_all.best_estimator_
# clf_adb.fit(features_trn, targets_trn)
# preds_tst = clf_adb.predict(features_tst)
# accu = np.mean(preds_tst==targets_tst)
# print('accu_tst_ADB', accu)
# print(classification_report(targets_tst, preds_tst))
# fpr, tpr, thresholds = metrics.roc_curve(targets_tst, preds_tst)
# print("%s, %s" %(fpr,tpr))
# plt.plot(fpr, tpr, label='ROC curve (area = %0.2f)' % metrics.auc(fpr, tpr))
# plt.plot([0, 1], [0, 1], color='navy', linestyle='--')
# plt.xlabel('FPR')
# plt.ylabel('TPR')
# plt.legend(loc="lower right")
# plt.show()

In [15]:
trials = 10
accu_tst_RFC = []
accu_tst_SVC = []
accu_tst_LR = []

for i in range(trials):
    features_trn, targets_trn = get_batch(n_size=5000, phase='train')
    features_tst, targets_tst = get_batch(n_size=1000, phase='test')

    clf = RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features=0.65, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=14, min_samples_split=7,
            min_weight_fraction_leaf=0.0, n_estimators=90, n_jobs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)
    clf.fit(features_trn, targets_trn)
    preds_tst = clf.predict(features_tst)
    accu = np.mean(preds_tst==targets_tst)
    # print('accu_tst_RFC', accu)
    accu_tst_RFC.append(accu)

    clf = SVC(gamma='auto')
    clf.fit(features_trn, targets_trn)
    preds_tst = clf.predict(features_tst)
    accu = np.mean(preds_tst==targets_tst)
    # print('accu_tst_SVC', accu)
    accu_tst_SVC.append(accu)

#     clf = LogisticRegression(solver='lbfgs', max_iter=200, multi_class='auto')
    clf = LogisticRegression(C=0.23357214690901212, class_weight=None, dual=False,
          fit_intercept=True, intercept_scaling=1, max_iter=100,
          multi_class='warn', n_jobs=None, penalty='l1', random_state=None,
          solver='liblinear', tol=0.0001, verbose=0, warm_start=False)
    clf.fit(features_trn, targets_trn)
    preds_tst = clf.predict(features_tst)
    accu = np.mean(preds_tst==targets_tst)
    # print('accu_tst_LR', accu)
    accu_tst_LR.append(accu)
    print('0% [',(i+1)*'.',(trials-i-1)*'-','] 100%', end='\r')
    
accu_tst_RFC = np.array(accu_tst_RFC)
accu_tst_SVC = np.array(accu_tst_SVC)
accu_tst_LR = np.array(accu_tst_LR)

print("\nDone")

0% [ ..........  ] 100%
Done


In [16]:
print(classification_report(targets_tst, preds_tst))

print('\nAverage accuracies after %s random trials:' %(trials))
print('accu_tst_RFC %.3f, accu_tst_SVC %.3f, accu_tst_LR %.3f'
      % (np.mean(accu_tst_RFC), np.mean(accu_tst_SVC), np.mean(accu_tst_LR)))

print('\nMax accuracies after %s random trials:' %(trials))
print('accu_tst_RFC %.3f, accu_tst_SVC %.3f, accu_tst_LR %.3f'
      % (np.max(accu_tst_RFC), np.max(accu_tst_SVC), np.max(accu_tst_LR)))

              precision    recall  f1-score   support

         0.0       0.79      0.87      0.83       500
         1.0       0.86      0.76      0.81       500

   micro avg       0.82      0.82      0.82      1000
   macro avg       0.82      0.82      0.82      1000
weighted avg       0.82      0.82      0.82      1000


Average accuracies after 10 random trials:
accu_tst_RFC 0.827, accu_tst_SVC 0.818, accu_tst_LR 0.821

Max accuracies after 10 random trials:
accu_tst_RFC 0.843, accu_tst_SVC 0.836, accu_tst_LR 0.836


### Accuracies from baseline: 
#### Cancer (ds.load_cancer()):
* accu_tst_RFC 0.758
* accu_tst_SVC 0.759
* accu_tst_LR 0.768

In [52]:
did_search = False
# best_clf, clf = None
if not did_search:
   # Create param grid.

    param_grid = [
        {'base_estimator':[clf_all.best_estimator_, DecisionTreeClassifier()],
        'n_estimators' : list(range(30,81,10)),
        'loss' : ['linear', 'square', 'exponential']}
    ]

    # Create grid search object

    clf = GridSearchCV(AdaBoostRegressor(), param_grid = param_grid, cv = 5, verbose=True, n_jobs=-1)
    # Fit on data
    features_val, targets_val = get_batch(n_size=5000, phase='validation')
    best_clf = clf.fit(features_val, targets_val)
    print(clf.best_params_)

Fitting 5 folds for each of 36 candidates, totalling 180 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:    4.3s
[Parallel(n_jobs=-1)]: Done 180 out of 180 | elapsed:  1.3min finished


{'base_estimator': Pipeline(memory=None,
     steps=[('classifier', LogisticRegression(C=0.23357214690901212, class_weight=None, dual=False,
          fit_intercept=True, intercept_scaling=1, max_iter=100,
          multi_class='warn', n_jobs=None, penalty='l1', random_state=None,
          solver='liblinear', tol=0.0001, verbose=0, warm_start=False))]), 'loss': 'linear', 'n_estimators': 60}


## Classification

In [76]:
# AdaboostRegressor
features_trn, targets_trn = get_batch(n_size=5000, phase='train')
features_tst, targets_tst = get_batch(n_size=1000, phase='test')
clf_adb = None
# if(clf is None):
base = LogisticRegression(C=0.23357214690901212, class_weight=None, dual=False,
          fit_intercept=True, intercept_scaling=1, max_iter=100,
          multi_class='warn', n_jobs=None, penalty='l1', random_state=None,
          solver='liblinear', tol=0.0001, verbose=0, warm_start=False)
clf_adb = AdaBoostRegressor(base_estimator=base, n_estimators=60, loss='linear')
# else:
#     clf_adb = clf.best_estimator_
    
clf_adb.fit(features_trn, targets_trn)
preds_tst = clf_adb.predict(features_tst)
accu = np.mean(preds_tst==targets_tst)
print('accu_tst_ADB', accu)
print(classification_report(targets_tst, preds_tst))

accu_tst_ADB 0.819
              precision    recall  f1-score   support

         0.0       0.79      0.87      0.83       500
         1.0       0.85      0.77      0.81       500

   micro avg       0.82      0.82      0.82      1000
   macro avg       0.82      0.82      0.82      1000
weighted avg       0.82      0.82      0.82      1000



In [ ]:
# Adaboost
features_trn, targets_trn = get_batch(n_size=5000, phase='train')
features_tst, targets_tst = get_batch(n_size=1000, phase='test')
clf = AdaBoostClassifier(n_estimators=100)
clf.fit(features_trn, targets_trn)
preds_tst = clf.predict(features_tst)
accu = np.mean(preds_tst==targets_tst)
print('accu_tst_ADB', accu)

### Calculate mutual information
*Caution* Very long computation, proceed with care
Note: disused because of enmormous memory requirements

In [ ]:
# dataset_features_sel_vals = dataset_features_sel.values
# dataset_targets_vals = np.ravel(dataset_targets.values) # unroll the column to the correct vector shape
# # dfs = [dataset_features_cont.copy(), dataset_features_disc.copy(), dataset_features_onehot.copy()]
# # retrieve values from dfs_out for subsequent steps if this comes after PCA
# # dataset_features_cont = dfs_out[0]
# # dataset_features_disc = dfs_out[1]
# # dataset_features_onehot = dfs_out[2]

# print('0% [---] 100%', end='\r')
# mutual_info_cont = mutual_info_regression(dataset_features_cont.values, dataset_targets_vals)
# print('0% [+--] 100%', end='\r')
# # mutual_info_disc = mutual_info_regression(dataset_features_disc.values, dataset_targets_vals)
# # print('0% [++-] 100%', end='\r')
# mutual_info_onehot = mutual_info_classif(dataset_features_onehot.values, dataset_targets_vals)
# print('0% [+++] 100%', end='\r')

### Convert mutual info into dataframes and inspect

In [ ]:
# if(mutual_info_cont.shape[0] != 1): # we only need to run this portion of the block once
#     mutual_info_cont = mutual_info_cont.reshape((1, len(mutual_info_cont)))
#     mutual_info_disc = mutual_info_disc.reshape((1, len(mutual_info_disc))) 
#     mutual_info_onehot = mutual_info_onehot.reshape((1, len(mutual_info_onehot))) 


# mutual_info_cont_df = pd.DataFrame(data=mutual_info_cont, columns=dataset_features_cont.columns)
# mutual_info_disc_df = pd.DataFrame(data=mutual_info_disc, columns=dataset_features_disc.columns)
# mutual_info_onehot_df = pd.DataFrame(data=mutual_info_onehot, columns=dataset_features_onehot.columns)

### Dropping features with low mutual information
    __Conditioned on target variable__

In [ ]:
# mi_dfs = [mutual_info_cont_df, mutual_info_disc_df, mutual_info_onehot_df]
# dfs = [dataset_features_cont, dataset_features_disc, dataset_features_onehot]
# dfs_out = []

# for i in range(len(dfs)):
#     mi = mi_dfs[i] # mutual information dataframe
#     df = dfs[i] # data containing dataframe
#     to_drop = []
#     mean = 0.25*mi.mean(axis=1)
    
#     # mi = mi.sort_values(by =0, axis=1) #sorts the values ascending
#     # print(mi)
#     for col in mi:
#         if(mi[col].iloc[0] < mean).all():
#             to_drop.append(col)
#     if(len(to_drop) > 0):
#         dfs_out.append(pd.DataFrame(df.drop(columns=to_drop)))
    
# dfs_out = pd.concat(dfs_out, axis=1)
# # Retained columns
# print(dfs_out.columns)
# # output shape
# print(dfs_out.shape)